In [ ]:
# Build fake readthedocs usage data

In [ ]:
import csv
import random
from datetime import datetime, timedelta

# Function to generate a random date within the specified range
def random_date(start_date, end_date):
    delta = end_date - start_date
    random_days = random.randint(0, delta.days)
    return start_date + timedelta(days=random_days)

# Function to generate fake website traffic data
def generate_traffic_data():
    start_date = datetime(2023, 3, 1)
    end_date = datetime(2023, 3, 31)
    versions = ['stable', 'latest']
    paths = [
        '/pages/index.html',
        '/pages/index.html',
        '/pages/index.html',
        '/pages/index.html',
        '/pages/index.html',
        '/pages/about.html',
        '/pages/contact.html',
        '/pages/features.html',
        '/pages/pricing.html',
        '/pages/get-started.html',
        '/pages/get-started.html',
        '/pages/get-started.html',
        '/pages/get-started.html',
        '/pages/get-started.html',
        '/pages/tutorial.html',
        '/pages/tutorial.html',
        '/pages/tutorial.html',
        '/pages/tutorial.html',
        '/pages/tutorial.html',
        '/pages/tutorial.html',
        '/pages/tutorial.html',
        '/pages/explore.html',
        '/pages/faqs.html',
        '/pages/faqs.html',
        '/pages/faqs.html',
        '/pages/faqs.html',
        '/pages/faqs.html',
        '/pages/contribution-guide.html',
        '/pages/team.html',
        '/pages/mission-statement.html',
        '/pages/our-values.html',
        '/pages/use-cases.html',
        '/pages/documentation.html',
        '/pages/directory.html',
        '/pages/developer-guide.html',
        '/pages/terms.html',
        '/pages/our-location.html',
        '/pages/branding-guidelines.html',
        '/pages/sitemap.html',
        '/pages/press.html',
        '/pages/news.html',
        
    ]

    data = []
    for _ in range(3000):
        date = random_date(start_date, end_date).strftime('%d/%m/%Y')
        version = random.choice(versions)
        path = random.choice(paths)
        views = random.randint(1, 40000)
        data.append([date, version, path, views])

    return data

# Generate fake website traffic data
traffic_data = generate_traffic_data()

# Write data to CSV file
with open('website_traffic_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Date', 'Version', 'Path', 'Views'])
    writer.writerows(traffic_data)

print("CSV file generated successfully.")

In [ ]:
# Vizro-AI usage

In [1]:
import pandas as pd

df = pd.read_csv('website_traffic_data.csv')
df.head()

,Date,Version,Path,Views
0,06/03/2023,stable,/pages/faqs.html,31771
1,06/03/2023,stable,/pages/news.html,17495
2,30/03/2023,stable,/pages/tutorial.html,22469
3,21/03/2023,stable,/pages/use-cases.html,23054
4,26/03/2023,latest,/pages/branding-guidelines.html,39206


In [ ]:
import vizro_ai
print(vizro_ai.__version__)
from dotenv import load_dotenv
load_dotenv()
from vizro_ai import VizroAI

vizro_ai = VizroAI()

In [ ]:
# Show total views by date
vizro_ai.plot(df, "Combine rows of Views for each Date and draw a line graph of Views per Date", explain=True)

In [ ]:
# Compare latest and stable by date
vizro_ai.plot(df, "Combine rows of Views for each Date for Version==latest and Version== stable. Draw a line graph comparing Views per Date for latest and stable", explain=True)

In [ ]:
# Total views of the top performing stable pages
vizro_ai.plot(df, "Collate rows of data for Path where Version==stable. Create a chart with bars for the total Views for the top 10 pages. Add the numbers to each bar", explain=True)

In [ ]:
# Top 5 performing pages on stable by date
vizro_ai.plot(df, "Collate rows of data for Path where Version==stable. Create a line graph for the total Views per Date for the top 5 Paths.", explain=True)


In [2]:
from vizro import Vizro
import vizro.models as vm

import vizro.plotly.express as px
import pandas as pd

df = pd.read_csv('website_traffic_data.csv')
df.head()

def views_custom_graph(data_frame):
    df = data_frame.groupby('Date')['Views'].sum().reset_index()
    fig = px.line(df, x='Date', y='Views')
    return fig

def views_separated_custom_barchart(data_frame):
    df_combined = data_frame.groupby(['Date', 'Version'])['Views'].sum().unstack()
    df = df_combined.reset_index()
    fig = px.line(df, x='Date', y=['latest', 'stable'], title='Views per Date for latest and stable')
    return fig

def popular_views_custom_barchart(data_frame, stable=True):
    if stable:
        df_version = data_frame[data_frame['Version'] == 'stable']
    else:
        df_version = data_frame[data_frame['Version'] == 'latest']

    chart_data = df_version.groupby('Path')['Views'].sum().nlargest(10)
    df = chart_data.reset_index()

    fig = px.bar(df, x='Path', y='Views', text='Views', title='Top 10 Pages by Views')
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
    return fig

def popular_views_custom_graph(data_frame, stable=True):
    if stable:
        df_version = data_frame[data_frame['Version'] == 'stable']
    else:
        df_version = data_frame[data_frame['Version'] == 'latest']

    df_version_grouped = df_version.groupby(['Date', 'Path']).sum().reset_index()
    top_5_paths = df_version_grouped.groupby('Path').sum().nlargest(5, 'Views').index
    df_top_5_paths = df_version_grouped[df_version_grouped['Path'].isin(top_5_paths)]
    df = df_top_5_paths.reset_index()

    fig = px.line(df_top_5_paths, x='Date', y='Views', color='Path')
    return fig

first_page = vm.Page(
title="Docs page data (last 90 days)",
layout=vm.Layout(grid=[[0,1],[0,1], [2,3], [2,3], [4,5],[4,5]]),
components=[
    # Total views by date
    vm.Graph(id='all views graph', figure=views_custom_graph(df)),
    
    # Latest and stable by date
    vm.Graph(id='version views graph', figure=views_separated_custom_barchart(df)),
    
    # Top performing stable pages
    vm.Graph(id='popular pages chart stable', figure=popular_views_custom_barchart(df)),

    # Top 5 stable pages by date
    vm.Graph(id='popular pages graph stable', figure=popular_views_custom_graph(df)),
    
    # Top performing latest pages
    vm.Graph(id='popular pages chart latest', figure=popular_views_custom_barchart(df, False)),

    # Top 5 latest pages by date
    vm.Graph(id='popular pages graph latest', figure=popular_views_custom_graph(df, False)),
    ],
)

In [ ]:
from vizro import Vizro
import vizro.models as vm
from vizro.models.types import capture

import vizro.plotly.express as px
import pandas as pd

df = pd.read_csv('website_traffic_data.csv')
df.head()

def views_custom_graph(data_frame):
    df = data_frame.groupby('Date')['Views'].sum().reset_index()
    fig = px.line(df, x='Date', y='Views')
    return fig

def views_separated_custom_barchart(data_frame):
    df_combined = data_frame.groupby(['Date', 'Version'])['Views'].sum().unstack()
    df = df_combined.reset_index()
    fig = px.line(df, x='Date', y=['latest', 'stable'], title='Views per Date for latest and stable')
    return fig

def popular_views_custom_barchart(data_frame, stable=True):
    if stable:
        df_version = data_frame[data_frame['Version'] == 'stable']
    else:
        df_version = data_frame[data_frame['Version'] == 'latest']

    chart_data = df_version.groupby('Path')['Views'].sum().nlargest(10)
    df = chart_data.reset_index()

    fig = px.bar(df, x='Path', y='Views', text='Views', title='Top 10 Pages by Views')
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
    return fig

def popular_views_custom_graph(data_frame, stable=True):
    if stable:
        df_version = data_frame[data_frame['Version'] == 'stable']
    else:
        df_version = data_frame[data_frame['Version'] == 'latest']

    df_version_grouped = df_version.groupby(['Date', 'Path']).sum().reset_index()
    top_5_paths = df_version_grouped.groupby('Path').sum().nlargest(5, 'Views').index
    df_top_5_paths = df_version_grouped[df_version_grouped['Path'].isin(top_5_paths)]
    df = df_top_5_paths.reset_index()

    fig = px.line(df_top_5_paths, x='Date', y='Views', color='Path')
    return fig

first_page = vm.Page(
title="Docs page data (last 90 days)",
layout=vm.Layout(grid=[[0,1],[0,1], [2,3], [2,3], [4,5],[4,5]]),
components=[
    # Total views by date
    vm.Graph(id='all views graph', figure=views_custom_graph(df)),
    
    # Latest and stable by date
    vm.Graph(id='version views graph', figure=views_separated_custom_barchart(df)),
    
    # Top performing stable pages
    vm.Graph(id='popular pages chart stable', figure=popular_views_custom_barchart(df)),

    # Top 5 stable pages by date
    vm.Graph(id='popular pages graph stable', figure=popular_views_custom_graph(df)),
    
    # Top performing latest pages
    vm.Graph(id='popular pages chart latest', figure=popular_views_custom_barchart(df, False)),

    # Top 5 latest pages by date
    vm.Graph(id='popular pages graph latest', figure=popular_views_custom_graph(df, False)),
    ],
)

In [3]:
dashboard = vm.Dashboard(pages=[first_page])
Vizro().build(dashboard).run()